# Discovering Ramanujan's Partition Congruences

The **partition function** $p(n)$ counts the number of ways to write $n$ as a sum of positive
integers (order does not matter). For example, $p(4) = 5$ because:

$$4 = 3+1 = 2+2 = 2+1+1 = 1+1+1+1$$

Ramanujan discovered three remarkable congruences:

- $p(5n+4) \equiv 0 \pmod{5}$
- $p(7n+5) \equiv 0 \pmod{7}$
- $p(11n+6) \equiv 0 \pmod{11}$

In this notebook, we use **q-Kangaroo** to compute partition values, generate
partition generating functions, and automatically discover these congruences.

In [1]:
from q_kangaroo import QSession, partition_count

# Compute partition values p(0) through p(15)
for n in range(16):
    print(f"p({n}) = {partition_count(n)}")

p(0) = 1
p(1) = 1
p(2) = 2
p(3) = 3
p(4) = 5
p(5) = 7
p(6) = 11
p(7) = 15
p(8) = 22
p(9) = 30
p(10) = 42
p(11) = 56
p(12) = 77
p(13) = 101
p(14) = 135
p(15) = 176


## Partition Generating Function

The partition generating function is the formal power series

$$\sum_{n=0}^{\infty} p(n)\, q^n = \prod_{k=1}^{\infty} \frac{1}{1 - q^k}$$

We can compute this directly as a truncated series:

In [2]:
from q_kangaroo import partition_gf

s = QSession()
pgf = partition_gf(s, 25)
pgf

1 + q + 2*q^2 + 3*q^3 + 5*q^4 + 7*q^5 + 11*q^6 + 15*q^7 + 22*q^8 + 30*q^9 + 42*q^10 + 56*q^11 + 77*q^12 + 101*q^13 + 135*q^14 + 176*q^15 + 231*q^16 + 297*q^17 + 385*q^18 + 490*q^19 + 627*q^20 + 792*q^21 + 1002*q^22 + 1255*q^23 + 1575*q^24 + O(q^25)

## Extracting Subsequences with `sift`

Ramanujan's first congruence says $p(5n+4) \equiv 0 \pmod{5}$. We can extract the
subsequence $p(5n+4)$ using `sift(f, m, j)`, which picks out the coefficient of
$q^{mn+j}$ from a series $f$:

In [3]:
from q_kangaroo import sift

# Compute partition_gf to high order, then extract p(5n+4)
pgf50 = partition_gf(s, 50)
sifted = sift(pgf50, 5, 4)
sifted

5 + 30*q + 135*q^2 + 490*q^3 + 1575*q^4 + 4565*q^5 + 12310*q^6 + 31185*q^7 + 75175*q^8 + 173525*q^9 + O(q^10)

Every coefficient (5, 30, 135, 490, 1575, ...) is divisible by 5!

## Automatic Congruence Discovery

The `findcong` function systematically searches for congruences of the form
$p(mn+r) \equiv 0 \pmod{d}$. Let's search with moduli 2, 3, 5, 7, 11:

In [4]:
from q_kangaroo import findcong

# Search for congruences with moduli [2, 3, 5, 7, 11]
pgf100 = partition_gf(s, 100)
congruences = findcong(pgf100, [2, 3, 5, 7, 11])

for c in congruences:
    print(f"p({c['modulus']}n + {c['residue']}) == 0 (mod {c['divisor']})")

p(5n + 4) == 0 (mod 5)
p(7n + 5) == 0 (mod 7)
p(11n + 6) == 0 (mod 11)


q-Kangaroo automatically rediscovers all three of Ramanujan's classical congruences.

## Verification

Let's verify the first congruence directly:

In [5]:
# Verify p(5n+4) mod 5 = 0 for n = 0, 1, ..., 9
for n in range(10):
    val = partition_count(5 * n + 4)
    print(f"p({5*n+4}) = {val}, mod 5 = {val % 5}")

p(4) = 5, mod 5 = 0
p(9) = 30, mod 5 = 0
p(14) = 135, mod 5 = 0
p(19) = 490, mod 5 = 0
p(24) = 1575, mod 5 = 0
p(29) = 4565, mod 5 = 0
p(34) = 12310, mod 5 = 0
p(39) = 31185, mod 5 = 0
p(44) = 75175, mod 5 = 0
p(49) = 173525, mod 5 = 0


## Historical Context

Ramanujan communicated these congruences to Hardy in 1919. The proofs rely on
deep properties of modular forms:

- The mod 5 and mod 7 congruences were proved by Ramanujan himself using
  properties of the Dedekind eta function.
- The mod 11 congruence required more sophisticated techniques, later
  developed by Atkin and Swinnerton-Dyer.

Dyson's **rank** of a partition, $\text{rank}(\lambda) = \text{largest part} - \text{number of parts}$,
explains the mod 5 and mod 7 congruences combinatorially. The **crank**, conjectured by
Dyson and defined by Andrews and Garvan, provides a unified combinatorial explanation
for all three congruences.